In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://www.tradingview.com/markets/stocks-usa/market-movers-high-dividend/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table', class_='table-Ngq2xrcG')
rows = table.find_all('tr', attrs={'data-rowkey': True})

data_rowkeys = [row['data-rowkey'] for row in rows]
print(data_rowkeys)

['NASDAQ:RILY', 'NASDAQ:IEP', 'NASDAQ:GIPR', 'NYSE:MNR', 'NYSE:TPVG', 'NASDAQ:OFS', 'NASDAQ:TCPC', 'NASDAQ:DALN', 'NASDAQ:RWAY', 'NASDAQ:ICON', 'NASDAQ:PTMN', 'NYSE:VOC', 'NASDAQ:XP', 'NASDAQ:TRIN', 'NASDAQ:OXSQ', 'NYSE:KEN', 'NASDAQ:GECC', 'NASDAQ:PSEC', 'NASDAQ:NHTC', 'NYSE:CATO', 'NYSE:NEP', 'NASDAQ:OCSL', 'NYSE:GSBD', 'NASDAQ:WHF', 'NASDAQ:SQFT', 'NYSE:FSK', 'NYSE:SAR', 'NYSE:CMTG', 'NASDAQ:FAT', 'NYSE:DSX', 'NASDAQ:MRCC', 'NYSE:CION', 'NASDAQ:MNDO', 'NYSE:TXO', 'NASDAQ:USEA', 'AMEX:COHN', 'NYSE:SCM', 'NASDAQ:WBA', 'NYSE:KRP', 'NASDAQ:DMLP', 'NYSE:NAT', 'NASDAQ:ARLP', 'NASDAQ:SLRC', 'NASDAQ:NMFC', 'NYSE:GHI', 'NYSE:AMBP', 'NASDAQ:HTCR', 'NYSE:PSBD', 'NYSE:BBDC', 'NYSE:PFLT', 'NYSE:SGHC', 'NYSE:BXSL', 'NASDAQ:GBDC', 'NYSE:UAN', 'NYSE:KSS', 'NYSE:BCSF', 'NYSE:OBDC', 'NASDAQ:TTEC', 'NYSE:CAPL', 'NYSE:DKL', 'NYSE:BSM', 'NASDAQ:BGFV', 'NYSE:EVC', 'NYSE:OBDE', 'NASDAQ:XRX', 'NASDAQ:SBLK', 'NASDAQ:MARPS', 'NASDAQ:CGBD', 'NYSE:SFL', 'NYSE:BWMX', 'NYSE:USAC', 'NASDAQ:ARCC', 'NASDAQ:CCAP', '

In [3]:
symbols = [item.split(':')[1] for item in data_rowkeys]
print(symbols)


['RILY', 'IEP', 'GIPR', 'MNR', 'TPVG', 'OFS', 'TCPC', 'DALN', 'RWAY', 'ICON', 'PTMN', 'VOC', 'XP', 'TRIN', 'OXSQ', 'KEN', 'GECC', 'PSEC', 'NHTC', 'CATO', 'NEP', 'OCSL', 'GSBD', 'WHF', 'SQFT', 'FSK', 'SAR', 'CMTG', 'FAT', 'DSX', 'MRCC', 'CION', 'MNDO', 'TXO', 'USEA', 'COHN', 'SCM', 'WBA', 'KRP', 'DMLP', 'NAT', 'ARLP', 'SLRC', 'NMFC', 'GHI', 'AMBP', 'HTCR', 'PSBD', 'BBDC', 'PFLT', 'SGHC', 'BXSL', 'GBDC', 'UAN', 'KSS', 'BCSF', 'OBDC', 'TTEC', 'CAPL', 'DKL', 'BSM', 'BGFV', 'EVC', 'OBDE', 'XRX', 'SBLK', 'MARPS', 'CGBD', 'SFL', 'BWMX', 'USAC', 'ARCC', 'CCAP', 'BGS', 'CSWC', 'TSLX', 'WES', 'TFSL', 'BRY', 'FDUS', 'OMF', 'CBRL', 'HIHO', 'EPM', 'SPOK', 'WLKP', 'LIEN', 'PAX', 'AB', 'SHIP', 'GLAD', 'DHT', 'MO', 'AY', 'CVI', 'DSWL', 'VTS', 'HTGC', 'WU', 'ET']


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Initialize an empty dictionary to store DataFrames for each stock
stock_data = {}

# Get data for each symbol
for symbol in symbols[0:10]:
    try:
        # Fetch the stock data
        stock = yf.Ticker(symbol)
        
        # Get dividend history
        dividends = stock.dividends

        # Create a DataFrame for this stock
        df = pd.DataFrame(dividends)
        df.columns = ['Dividend']
        df.index.name = 'Date'

        # Add columns for stock price 3 days before and after dividend
        df['Price_Before'] = None
        df['Price_After'] = None

        for date in df.index:
            # Get stock price 3 days before dividend
            before_date = date - timedelta(days=3)
            history_before = stock.history(start=before_date, end=date)
            if not history_before.empty:
                df.loc[date, 'Price_Before'] = history_before['Close'].iloc[-1]

            # Get stock price 3 days after dividend
            after_date = date + timedelta(days=3)
            history_after = stock.history(start=date, end=after_date)
            if not history_after.empty:
                df.loc[date, 'Price_After'] = history_after['Close'].iloc[-1]

        # Add this DataFrame to the dictionary
        stock_data[symbol] = df

        print(f"Processed {symbol}")
        
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")

# Display the resulting dictionary of DataFrames
for symbol, df in stock_data.items():
    print(f"\nData for {symbol}:")
    print(df)


Processed RILY


$IEP: possibly delisted; no price data found  (1d 1989-03-24 00:00:00-05:00 -> 1989-03-27 00:00:00-05:00)
$IEP: possibly delisted; no price data found  (1d 1993-12-24 00:00:00-05:00 -> 1993-12-27 00:00:00-05:00)
$IEP: possibly delisted; no price data found  (1d 2013-02-16 00:00:00-05:00 -> 2013-02-19 00:00:00-05:00)


Processed IEP
Processed GIPR
Processed MNR
Processed TPVG
Processed OFS
Processed TCPC
Processed DALN
Processed RWAY
Processed ICON

Data for RILY:
                           Dividend Price_Before Price_After
Date                                                        
2014-11-14 00:00:00-05:00  0.029703     4.541428    4.147873
2015-05-20 00:00:00-04:00  0.059406     5.696206    5.623309
2015-11-20 00:00:00-05:00  0.059406      4.94643    5.103625
2016-08-18 00:00:00-04:00  0.029703     4.935951    5.004286
2016-11-25 00:00:00-05:00  0.250000     8.962507    9.188202
2017-03-02 00:00:00-05:00  0.260000     8.406803    8.138268
2017-05-19 00:00:00-04:00  0.160000     7.699843    7.649951
2017-08-17 00:00:00-04:00  0.130000     8.758638    8.579779
2017-11-21 00:00:00-05:00  0.120000    10.088923    10.52219
2018-03-19 00:00:00-04:00  0.160000    10.916066   11.148688
2018-05-18 00:00:00-04:00  0.120000    11.829524   11.812405
2018-08-15 00:00:00-04:00  0.300000    12.982232   12.40396

In [5]:
# Initialize a dictionary to store the capital history for each symbol
capital_histories = {}

# Define start and end dates
start_date = pd.to_datetime('2018-01-01', utc=True)
end_date = pd.to_datetime('2024-10-01', utc=True)

# Iterate through each symbol and its corresponding DataFrame
for symbol, df in stock_data.items():
    capital = 10000  # Starting capital for each symbol
    capital_history = []  # List to store capital values over time
    
    # Filter DataFrame for the specified date range
    df_filtered = df.loc[start_date:end_date]
    
    # Iterate through each dividend date
    for date, row in df_filtered.iterrows():
        if pd.isna(row['Price_Before']) or pd.isna(row['Price_After']) or pd.isna(row['Dividend']):
            continue  # Skip if we don't have price data or dividend data
        
        # Calculate number of shares we can buy
        shares = capital // row['Price_Before']
        
        # Calculate new capital after buying shares
        capital_after_buy = capital - (shares * row['Price_Before'])
        
        # Calculate capital after dividend payout
        capital_after_dividend = capital_after_buy + (shares * row['Dividend'])
        
        # Calculate final capital after selling shares
        capital = capital_after_dividend + (shares * row['Price_After'])
        
        # Store the capital value for this date
        capital_history.append((date, capital))
    
    # Store the capital history for this symbol
    capital_histories[symbol] = capital_history

# Create a DataFrame for each symbol's capital history
for symbol, history in capital_histories.items():
    df = pd.DataFrame(history, columns=['Date', 'Capital'])
    df.set_index('Date', inplace=True)
    capital_histories[symbol] = df

# Print the capital history for each symbol
for symbol, df in capital_histories.items():
    print(f"\nCapital history for {symbol}:")
    print(df)

# Calculate and print final capitals
final_capitals = {}
for symbol, df in capital_histories.items():
    if not df.empty:
        final_capitals[symbol] = df['Capital'].iloc[-1]
    else:
        print(f"Warning: No capital history data for {symbol}")
        final_capitals[symbol] = 10000  # Use initial capital if no data

sorted_capitals = sorted(final_capitals.items(), key=lambda x: x[1], reverse=True)

print("\nFinal capital for each symbol, ranked:")
for symbol, capital in sorted_capitals:
    print(f"{symbol}: ${capital:.2f}")


Capital history for RILY:
                                Capital
Date                                   
2018-03-19 00:00:00-04:00  10359.641886
2018-05-18 00:00:00-04:00  10449.662405
2018-08-15 00:00:00-04:00  10225.932591
2018-11-15 00:00:00-05:00  10611.520581
2019-03-18 00:00:00-04:00  10621.754014
2019-05-14 00:00:00-04:00  11217.626028
2019-08-14 00:00:00-04:00  11826.973619
2019-11-13 00:00:00-05:00  12263.769774
2020-03-16 00:00:00-04:00   9759.393954
2020-05-29 00:00:00-04:00   9998.392956
2020-08-13 00:00:00-04:00  10155.013830
2020-11-09 00:00:00-05:00  10832.534266
2021-03-09 00:00:00-05:00  12567.826269
2021-05-14 00:00:00-04:00  13117.889265
2021-08-12 00:00:00-04:00  13430.036082
2021-11-08 00:00:00-05:00  13687.542261
2022-03-08 00:00:00-05:00  14434.651743
2022-05-10 00:00:00-04:00  14533.865111
2022-08-10 00:00:00-04:00  15781.044340
2022-11-14 00:00:00-05:00  15476.209257
2023-03-09 00:00:00-05:00  13086.279486
2023-05-15 00:00:00-04:00  14694.042505
2023-08-10 00

In [6]:
stock_data

{'RILY':                            Dividend Price_Before Price_After
 Date                                                        
 2014-11-14 00:00:00-05:00  0.029703     4.541428    4.147873
 2015-05-20 00:00:00-04:00  0.059406     5.696206    5.623309
 2015-11-20 00:00:00-05:00  0.059406      4.94643    5.103625
 2016-08-18 00:00:00-04:00  0.029703     4.935951    5.004286
 2016-11-25 00:00:00-05:00  0.250000     8.962507    9.188202
 2017-03-02 00:00:00-05:00  0.260000     8.406803    8.138268
 2017-05-19 00:00:00-04:00  0.160000     7.699843    7.649951
 2017-08-17 00:00:00-04:00  0.130000     8.758638    8.579779
 2017-11-21 00:00:00-05:00  0.120000    10.088923    10.52219
 2018-03-19 00:00:00-04:00  0.160000    10.916066   11.148688
 2018-05-18 00:00:00-04:00  0.120000    11.829524   11.812405
 2018-08-15 00:00:00-04:00  0.300000    12.982232   12.403961
 2018-11-15 00:00:00-05:00  0.160000    10.119781   10.341551
 2019-03-18 00:00:00-04:00  0.080000     9.991388    9.921024


In [22]:
# Define your start and end dates
start_date = pd.Timestamp('2021-01-01').date()
end_date = pd.Timestamp('2024-10-01').date()

# Create a date range and initialize the centralized DataFrame
date_range = pd.date_range(start=start_date, end=end_date).date
central_df = pd.DataFrame(index=date_range)

# Process each stock's DataFrame and merge it into the centralized DataFrame
for symbol, df in stock_data.items():
    
    # Convert the index to date objects
    df.index = pd.to_datetime(df.index).date

    # Select the 'Dividend' column
    dividend_series = df['Dividend']
    
    # Reindex to match the central DataFrame's index
    dividend_series = dividend_series.reindex(central_df.index)
    
    # Add the dividend series to the central DataFrame
    central_df[symbol] = dividend_series

# Display the entire centralized DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(central_df)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')


            RILY  IEP   GIPR   MNR  TPVG   OFS  TCPC  DALN  RWAY  ICON
2021-01-01   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-02   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-03   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-04   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-05   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-06   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-07   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-08   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-09   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-10   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-11   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-12   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-01-13   NaN  NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
2021-0

In [44]:
import pandas as pd
import yfinance as yf
from datetime import timedelta
import pandas_market_calendars as mcal
import warnings

# Suppress warnings from yfinance about missing data
warnings.filterwarnings("ignore")

# Define your start and end dates
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2023-10-01')

# List of stock symbols (use known dividend-paying stocks)
symbols = ['AAPL', 'MSFT', 'T', 'VZ', 'JNJ']

# Get NYSE trading calendar
nyse = mcal.get_calendar('NYSE')
trading_days = nyse.valid_days(start_date=start_date, end_date=end_date)

# Convert trading_days to a set for faster lookup
trading_days_set = set(trading_days)

# Function to get the previous trading day
def get_previous_trading_day(date):
    min_date = trading_days[0]
    while date not in trading_days_set:
        date -= timedelta(days=1)
        if date < min_date:
            # No previous trading day available
            return None
    return date

# Function to get the next trading day
def get_next_trading_day(date):
    max_date = trading_days[-1]
    while date not in trading_days_set:
        date += timedelta(days=1)
        if date > max_date:
            # No next trading day available
            return None
    return date

# Initialize an empty list to collect trades
trades = []

# Fetch data for each stock
for symbol in symbols:
    print(f"\nProcessing {symbol}...")
    # Fetch the stock history with dividends and stock splits
    stock = yf.Ticker(symbol)
    try:
        history = stock.history(start=start_date, end=end_date, actions=True, auto_adjust=False)
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        continue

    # Check if history is empty
    if history.empty:
        print(f"No trading data available for {symbol}. Skipping.")
        continue

    # Extract dividend events
    dividends = history[history['Dividends'] != 0]['Dividends']

    # Skip if there are no dividends
    if dividends.empty:
        print(f"No dividends found for {symbol}. Skipping.")
        continue
    else:
        print(f"Found {len(dividends)} dividends for {symbol}")

    # For each dividend event, determine buy and sell dates
    for ex_div_date, dividend_amount in dividends.items():
        # Ensure ex_div_date is a Timestamp
        if not isinstance(ex_div_date, pd.Timestamp):
            ex_div_date = pd.Timestamp(ex_div_date)
        
        print(f"\nProcessing dividend for {symbol} on ex-div date {ex_div_date.date()} with amount {dividend_amount}")
        
        # Define buy date (2 business days before ex-dividend date)
        buy_date = ex_div_date - pd.Timedelta(days=2)
        buy_date = get_previous_trading_day(buy_date)
        if buy_date is None:
            print(f"No trading day available before ex-dividend date for {symbol}. Skipping trade.")
            continue
        print(f"Calculated buy_date: {buy_date.date()}")

        # Define sell date (2 business days after ex-dividend date)
        sell_date = ex_div_date + pd.Timedelta(days=2)
        sell_date = get_next_trading_day(sell_date)
        if sell_date is None:
            print(f"No trading day available after ex-dividend date for {symbol}. Skipping trade.")
            continue
        print(f"Calculated sell_date: {sell_date.date()}")

        # Get price at buy date
        try:
            price_at_buy = history.loc[buy_date]['Close']
        except KeyError:
            print(f"Buy date {buy_date.date()} not in trading data for {symbol}. Skipping trade.")
            continue

        # Calculate dividend yield
        dividend_yield = dividend_amount / price_at_buy

        # Append the trade details to the trades list
        trades.append({
            'symbol': symbol,
            'buy_date': buy_date,
            'sell_date': sell_date,
            'ex_div_date': ex_div_date,
            'dividend_amount': dividend_amount,
            'price_at_buy': price_at_buy,
            'dividend_yield': dividend_yield
        })
        print(f"Trade added for {symbol}: Buy on {buy_date.date()}, Sell on {sell_date.date()}, Dividend Yield: {dividend_yield:.4%}")

# Convert trades list to DataFrame
trades_df = pd.DataFrame(trades)

print(f"\nTotal trades collected: {len(trades)}")

# Check if trades_df is empty
if trades_df.empty:
    print("No trades were collected. Exiting the simulation.")
    exit()
else:
    # Sort trades by buy_date
    trades_df.sort_values(by='buy_date', inplace=True)

# The rest of your simulation code...



Processing AAPL...
Found 11 dividends for AAPL

Processing dividend for AAPL on ex-div date 2021-02-05 with amount 0.205
No trading day available before ex-dividend date for AAPL. Skipping trade.

Processing dividend for AAPL on ex-div date 2021-05-07 with amount 0.22
No trading day available before ex-dividend date for AAPL. Skipping trade.

Processing dividend for AAPL on ex-div date 2021-08-06 with amount 0.22
No trading day available before ex-dividend date for AAPL. Skipping trade.

Processing dividend for AAPL on ex-div date 2021-11-05 with amount 0.22
No trading day available before ex-dividend date for AAPL. Skipping trade.

Processing dividend for AAPL on ex-div date 2022-02-04 with amount 0.22
No trading day available before ex-dividend date for AAPL. Skipping trade.

Processing dividend for AAPL on ex-div date 2022-05-06 with amount 0.23
No trading day available before ex-dividend date for AAPL. Skipping trade.

Processing dividend for AAPL on ex-div date 2022-08-05 with am

: 